In [1]:
import calendar
import os
import sys
from datetime import datetime
from sqlalchemy.sql import extract
from collections import defaultdict, OrderedDict
from sqlalchemy import create_engine
caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'ajna_docs', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from bhadrasana.models import engine, db_session
from bhadrasana.models import Usuario
from bhadrasana.models.ovr import OVR, EventoOVR

Fazendo log de erros e alertas no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ..\..\ajna_docs\commons\ajna_commons\flask\access.log


In [20]:
def get_fichas_tipo(ano, mes):
    lista_eventos = db_session.query(EventoOVR).filter(EventoOVR.fase >= 3)\
                                               .filter(extract('year', EventoOVR.create_date) ==  ano)\
                                               .filter(extract('month', EventoOVR.create_date) ==  mes)\
                                               .all()
    ficha_tipo = defaultdict(list)
    ovrs_recuperadas = set()
    for evento in lista_eventos:
        ovr = db_session.query(OVR).filter(OVR.id == evento.ovr_id).one()
        if (ovr.setor_id not in ('1', '2', '3')) or (ovr.id in ovrs_recuperadas):
            continue
        ovrs_recuperadas.add(ovr.id)
        ficha_tipo[ovr.tipooperacao].append(ovr)
    return ficha_tipo

In [21]:
def dump_model(model, exclude: list = None):
    return dict([(k, v) for k, v in vars(model).items() if not k.startswith('_')])

In [22]:
def preenche_linha_ovr(tipooperacao, ano, mes):
    inicio = f'01/{mes}/{ano}'
    lastday = calendar.monthrange(ano, mes)[1]
    termino = f'{lastday}/{mes}/{ano}'
    campos_ovr = OrderedDict()
    NOMES_TIPOOPERACAO = {
        1: f'IMPORTAÇÃO - OPERAÇÃO LUNETA {ano}{mes}',
        # Observação: ideal seria dividir mais as OVRs, pois além de Operação Luneta, tivemos Outlet, Ramenta, temos a seleção de 
        # bagagem, etc. Mas aí precisaria ser "Hard-coded" o agrupamento por OVRs
        2: f'EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO {ano}{mes}',
        # Observação: hoje não temos, mas se fizermos OUTROS TIPOS DE OPERAÇÕES em exportação que não a habitual busca de cocaína,
        # o ideal seria separar em outras OVRs
        3: f'IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO LUNETA {ano}{mes}',
        # Observação: ideal seria dividir mais as OVRs, pois além de Operação Luneta, tivemos Outlet, Ramenta, temos a seleção de 
        # bagagem, etc. Mas aí precisaria ser "Hard-coded" o agrupamento por OVRs
        4: f'EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO PORTO BLINDADO {ano}{mes}',
        # Observação: existe também o código "Pós-incidente" e não temos distinção no Fichas, vamos criar um Flag.
        # no caso deste flag, substituir estes campos por 'Tipo': 'Pós-Incidente - Portos', 'Código': '30.02.10'
        5: f'VIGILÂNCIA - OPERAÇÃO REDOMA {ano}{mes}',
    }
    campos_ovr['Nome'] = NOMES_TIPOOPERACAO[tipooperacao]
    if tipooperacao in (1, 2):
        campos_ovr['Modalidade'] = 'Vigilância e Repressão'
        campos_ovr['Tipo'] = 'Portos'
        campos_ovr['Código'] = '20.00.10'
    elif tipooperacao in (3, 4):
        campos_ovr['Modalidade'] = 'Pesquisa e Seleção'
        campos_ovr['Tipo'] = 'Pré-Incidente - Portos'
        campos_ovr['Código'] = '30.01.10'
    elif tipooperacao == 7:
        campos_ovr['Modalidade'] = 'Vigilância e Repressão'
        # Observação: Secta e-OVR não possui a tipificação de vigilância/rondas, auditorias e conformidade
        # Utilizando este tipo e código para distinção
        campos_ovr['Tipo'] = 'Ponto de fronteira'
        campos_ovr['Código'] = '20.00.09'
    else:
        raise Exception(f'tipooperacao {tipooperacao} não previsto!!')
    
    campos_ovr['Abrangência'] = 'Local'
    campos_ovr['Início'] = inicio
    campos_ovr['Término'] = termino
    campos_ovr['Local da Execução'] = 'Porto de Santos'
    campos_ovr['Coordenador'] = 'Ivan da Silva Brasílico'
    campos_ovr['Coordenador_CPF'] = '25052288840'
    campos_ovr['Motivação'] = 'Demanda Interna'
    return campos_ovr

In [27]:
def preenche_linha_operacao(operacao, modalidade):
    linha = OrderedDict()
    linha['Órgão'] = 'RFB'
    linha['Equipe'] = operacao.setor.nome
    linha['Modalidade Operação'] = modalidade
    linha['Tipo'] = 'Portos'
    linha['Local da Operação'] = operacao.recinto.nome
    linha['Documentos de origem'] = f'Ficha {operacao.id}'
    linha['Ocorrências durante a operação'] = operacao.observacoes
    linha['Supervisor da RFB'] = operacao.cpfauditorresponsavel
    linha['Responsável'] = operacao.responsavel_cpf
    servidores = set()
    for evento in operacao.historico:
        # servidor = db_session.query(Usuario).filter(Usuario.cpf == evento.user_name).one()
        servidores.add(evento.user_name)
    linha['Servidores'] = servidores
    return linha


In [28]:
mes = 3
ano = 2023

fichas_tipo = get_fichas_tipo(ano, mes)

In [30]:
lista_final = []
for tipooperacao, fichas in fichas_tipo.items():
    print(tipooperacao, len(fichas))
    try:
        linha_ovr = preenche_linha_ovr(tipooperacao, ano, mes)
    except Exception as err:
        print(err)
        continue
    for ficha in fichas:
        linha_operacao = preenche_linha_operacao(ficha, linha_ovr['Modalidade'])
        linha_final = linha_ovr.copy()
        linha_final.update(linha_operacao)
        lista_final.append(linha_final)

1 17
2 158
7 21
7
6 1
6
3 7
5 2
tipooperacao 5 não previsto!!
0 2
0


In [32]:
import pandas as pd
df = pd.DataFrame(lista_final)

In [35]:
df

,Nome,Modalidade,Tipo,Código,Abrangência,Início,Término,Local da Execução,Coordenador,Coordenador_CPF,Motivação,Órgão,Equipe,Modalidade Operação,Local da Operação,Documentos de origem,Ocorrências durante a operação,Supervisor da RFB,Responsável,Servidores
0,IMPORTAÇÃO - OPERAÇÃO LUNETA 20233,Vigilância e Repressão,Portos,20.00.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Vigilância e Repressão,BANDEIRANTES DEICMAR - IPA OUTEIRINHOS,Ficha 2227,Bagagem desacompanahda. Suspeita de descaminho...,25052288840,06806235824,"{02768829544, 13397296870, 25052288840, 068062..."
1,IMPORTAÇÃO - OPERAÇÃO LUNETA 20233,Vigilância e Repressão,Portos,20.00.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 4833,None,02609824904,02609824904,"{08937951860, 22983267890, 02609824904, 133972..."
2,IMPORTAÇÃO - OPERAÇÃO LUNETA 20233,Vigilância e Repressão,Portos,20.00.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 2535,CE 122105193596920 - 2021-07-21 - USNYC\r\nCPF...,06806235824,06806235824,"{09720242809, 13397296870, 25052288840, 068062..."
3,IMPORTAÇÃO - OPERAÇÃO LUNETA 20233,Vigilância e Repressão,Portos,20.00.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 4876,None,02609824904,02609824904,"{08937951860, 13397296870, 02609824904}"
4,IMPORTAÇÃO - OPERAÇÃO LUNETA 20233,Vigilância e Repressão,Portos,20.00.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Vigilância e Repressão,TERMARES IPA,Ficha 4906,None,02609824904,02609824904,"{09720242809, 02609824904, 13397296870}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,Pesquisa e Seleção,Portos,30.01.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Pesquisa e Seleção,ALFANDEGA,Ficha 5829,Análise de risco da bagagem desacompanhada de ...,None,None,{25945585831}
178,IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,Pesquisa e Seleção,Portos,30.01.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Pesquisa e Seleção,ALFANDEGA,Ficha 5830,Análise de risco da bagagem desacompanhada de ...,None,None,{25945585831}
179,IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,Pesquisa e Seleção,Portos,30.01.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Pesquisa e Seleção,ALFANDEGA,Ficha 5831,Análise de risco da bagagem desacompanhada de ...,None,None,{25945585831}
180,IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,Pesquisa e Seleção,Portos,30.01.10,Local,01/3/2023,31/3/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,Demanda Interna,RFB,ALFSTS/DIREP/EQRIMP,Pesquisa e Seleção,ALFANDEGA,Ficha 5832,Análise de risco da bagagem desacompanhada de ...,None,None,{25945585831}


In [36]:
df.to_excel('teste.xlsx')